# Hybrid CatBoost–SARIMAX Framework for Multi-City AQI Prediction

This study proposes a hybrid modeling framework combining CatBoost regression
with SARIMAX residual modeling for daily AQI prediction across six major Indian cities.

The objective is to capture nonlinear pollutant interactions (CatBoost)
alongside residual temporal patterns (SARIMAX).

In [ ]:
!pip install catboost lightgbm xgboost

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

DATA_FOLDER = "/content/drive/MyDrive/AQI_Hybrid_Project/city_data/"
print(os.listdir(DATA_FOLDER))

In [ ]:
import os
import glob
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit

from statsmodels.tsa.statespace.sarimax import SARIMAX

import itertools
import math

sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 110

In [ ]:
csv_files = sorted(glob.glob(os.path.join(DATA_FOLDER, "*.csv")))
print("Found files:", csv_files)

city_dfs = {}

for fp in csv_files:
    city = os.path.splitext(os.path.basename(fp))[0]
    df = pd.read_csv(fp)

    df.columns = [c.strip() for c in df.columns]

    # Timestamp
    if "Date" in df.columns:
        df["Timestamp"] = pd.to_datetime(df["Date"], errors="coerce")
    else:
        raise ValueError(f"No Date column found in {city}")

    df = df.dropna(subset=["Timestamp"])
    df = df.sort_values("Timestamp").set_index("Timestamp")

    # Convert numeric columns
    for col in df.columns:
        if col != "Date":
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # ❗ Remove rows where ALL pollutant values are missing
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df = df.dropna(subset=numeric_cols, how="all")

    # Forward fill
    df = df.ffill()

    # Median fill remaining
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

    city_dfs[city] = df
    print(f"{city}: rows={len(df)}")

In [ ]:
city_dfs_continuous = {}

for city, df in city_dfs.items():

    df = df.copy()

    # Create full daily date range
    full_range = pd.date_range(
        start=df.index.min(),
        end=df.index.max(),
        freq="D"
    )

    # Reindex to daily frequency
    df = df.reindex(full_range)

    # Forward fill
    df = df.ffill()

    # Median fill remaining numeric
    df = df.fillna(df.median(numeric_only=True))

    city_dfs_continuous[city] = df

    print(f"{city}: After reindex -> rows={len(df)}")

In [ ]:
results_store = {}
for city, df in city_dfs_continuous.items():

    if "AQI" not in df.columns:
        print(f"{city}: AQI not found — skipping")
        continue

    df = df.dropna(subset=["AQI"]).copy()

    results_store[city] = {
        "df": df,
        "TARGET": "AQI"
    }

    print(f"{city}: Using AQI as target | Rows = {len(df)}")

In [ ]:
def cap_outliers_iqr(series, threshold=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - threshold * IQR
    upper = Q3 + threshold * IQR
    return series.clip(lower, upper)


def create_time_series_features(df, target):

    df = df.copy()

    # Calendar features
    df["dayofweek"] = df.index.dayofweek
    df["month"] = df.index.month
    df["year"] = df.index.year

    # Lags
    for lag in [1,2,3,7,14]:
        df[f"{target}_lag{lag}"] = df[target].shift(lag)

    # Rolling mean
    df[f"{target}_roll7"] = df[target].shift(1).rolling(7).mean()

    df = df.dropna()
    return df


city_features = {}

for city, meta in results_store.items():

    df = meta["df"].copy()

    # Cap outliers (excluding AQI)
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if "AQI" in num_cols:
        num_cols.remove("AQI")

    for col in num_cols:
        df[col] = cap_outliers_iqr(df[col])

    df_feat = create_time_series_features(df, "AQI")

    city_features[city] = df_feat

    print(f"{city}: rows after feature engineering = {len(df_feat)}")

In [ ]:
splits_store = {}

for city, df in city_features.items():

    X = df.select_dtypes(include=[np.number]).drop(columns=["AQI"])
    y = df["AQI"]

    split_index = int(len(df) * 0.8)

    X_train = X.iloc[:split_index]
    X_test  = X.iloc[split_index:]
    y_train = y.iloc[:split_index]
    y_test  = y.iloc[split_index:]

    splits_store[city] = {
        "X_train": X_train,
        "X_test": X_test,
        "y_train": y_train,
        "y_test": y_test
    }

    print(f"{city}: Train={len(X_train)}, Test={len(X_test)}")

In [ ]:
models_store = {}

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import math

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

for city, data in splits_store.items():

    print(f"\nTraining CatBoost for {city}")

    X_train = data["X_train"]
    X_test  = data["X_test"]
    y_train = data["y_train"]
    y_test  = data["y_test"]

    model = CatBoostRegressor(
        iterations=2000,
        learning_rate=0.03,
        depth=6,
        loss_function="RMSE",
        random_seed=42,
        verbose=200,
        early_stopping_rounds=100
    )

    model.fit(
        X_train,
        y_train,
        eval_set=(X_test, y_test),
        use_best_model=True
    )

    pred_train = model.predict(X_train)
    pred_test  = model.predict(X_test)

    rmse_cb = rmse(y_test, pred_test)
    r2_cb = r2_score(y_test, pred_test)

    print(f"{city} — CatBoost RMSE: {rmse_cb:.2f}, R2: {r2_cb:.3f}")

    models_store[city] = {
        "model": model,
        "y_train": y_train,
        "y_test": y_test,
        "pred_train": pred_train,
        "pred_test": pred_test
    }

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
import numpy as np

for city, data in models_store.items():

    residuals = data["y_test"] - data["pred_test"]

    print(f"\nResidual analysis for {city}")

    plt.figure(figsize=(12,4))
    plt.plot(residuals)
    plt.title(f"{city} — CatBoost Residuals")
    plt.show()

    plt.figure(figsize=(10,4))
    plot_acf(residuals, lags=40)
    plt.title(f"{city} — Residual ACF")
    plt.show()

from statsmodels.stats.diagnostic import acorr_ljungbox

lb = acorr_ljungbox(residuals, lags=[10], return_df=True)
print("Ljung-Box p-value:", lb["lb_pvalue"].values[0])

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
import numpy as np

hybrid_store = {}

for city, data in models_store.items():

    print(f"\nHybrid modeling for {city}")

    y_train = data["y_train"]
    y_test  = data["y_test"]
    pred_train = data["pred_train"]
    pred_test  = data["pred_test"]

    residual_train = y_train - pred_train

    try:
        sarimax = SARIMAX(
            residual_train,
            order=(1,1,1),
            seasonal_order=(1,0,1,7),
            enforce_stationarity=False,
            enforce_invertibility=False
        ).fit(disp=False)

        resid_forecast = sarimax.forecast(steps=len(y_test))
        resid_forecast = pd.Series(resid_forecast, index=y_test.index)

    except Exception as e:
        print(f"{city} — SARIMAX failed, using zero residuals. Error: {e}")
        resid_forecast = pd.Series(np.zeros(len(y_test)), index=y_test.index)

    hybrid_pred = pred_test + resid_forecast

    rmse_hybrid = rmse(y_test, hybrid_pred)
    r2_hybrid = r2_score(y_test, hybrid_pred)

    print(f"{city} — Hybrid RMSE: {rmse_hybrid:.2f}, R2: {r2_hybrid:.3f}")

    hybrid_store[city] = {
        "hybrid_pred": hybrid_pred
    }

In [ ]:
from sklearn.metrics import mean_absolute_error

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

results = []

for city in models_store:

    y_test = models_store[city]["y_test"]
    cb_pred = models_store[city]["pred_test"]
    hybrid_pred = hybrid_store[city]["hybrid_pred"]

    results.append({
        "City": city,
        "Model": "CatBoost",
        "RMSE": rmse(y_test, cb_pred),
        "MAE": mean_absolute_error(y_test, cb_pred),
        "MAPE": mape(y_test, cb_pred),
        "R2": r2_score(y_test, cb_pred)
    })

    results.append({
        "City": city,
        "Model": "Hybrid",
        "RMSE": rmse(y_test, hybrid_pred),
        "MAE": mean_absolute_error(y_test, hybrid_pred),
        "MAPE": mape(y_test, hybrid_pred),
        "R2": r2_score(y_test, hybrid_pred)
    })

comparison_df = pd.DataFrame(results)
comparison_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.barplot(data=comparison_df, x="City", y="RMSE", hue="Model")
plt.title("RMSE Comparison Across Cities")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

for city, data in models_store.items():

    y_test = data["y_test"]
    pred_test = data["pred_test"]

    plt.figure(figsize=(12,5))
    plt.plot(y_test.index, y_test, label="Actual", color="black")
    plt.plot(y_test.index, pred_test, label="CatBoost", color="blue", alpha=0.8)

    plt.title(f"{city} — Actual vs CatBoost")
    plt.xlabel("Time")
    plt.ylabel("AQI")
    plt.legend()
    plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
import numpy as np

for city, data in models_store.items():

    residuals = data["y_test"] - data["pred_test"]

    plt.figure(figsize=(10,4))
    plot_acf(residuals, lags=40)
    plt.title(f"{city} — Residual ACF (CatBoost)")
    plt.show()

In [ ]:
for city in models_store:

    y_test = models_store[city]["y_test"]
    cb_pred = models_store[city]["pred_test"]
    hybrid_pred = hybrid_store[city]["hybrid_pred"]

    plt.figure(figsize=(12,5))
    plt.plot(y_test.index, y_test, label="Actual", color="black")
    plt.plot(y_test.index, cb_pred, label="CatBoost", color="blue")
    plt.plot(y_test.index, hybrid_pred, label="Hybrid", color="red", alpha=0.8)

    plt.title(f"{city} — CatBoost vs Hybrid")
    plt.legend()
    plt.show()

## Conclusion

The hybrid CatBoost–SARIMAX framework was evaluated across six Indian metropolitan cities.

Results indicate that CatBoost alone captures most nonlinear and temporal
patterns in AQI data. Hybrid modeling provided marginal improvements in select cities,
suggesting limited residual autocorrelation after primary modeling.

This demonstrates that tree-based boosting models, when combined with
lag-based feature engineering, can effectively model AQI dynamics
without requiring extensive secondary temporal correction.